In [44]:
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw 
from os import listdir
from os.path import isfile, join
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

export_path = "models/bce_20220310_222556"

model = tf.compat.v1.keras.experimental.load_from_saved_model(export_path, custom_objects={'KerasLayer':hub.KerasLayer})

IMG_SIZE = 224 # Specify height and width of image to match the input format of the model
CHANNELS = 3 # Keep RGB color channels to match the input format of the model


def show_prediction(title, model, classes):
    
    # Get movie info
    img_path = os.path.join('multi_class_images/', title)
    img1 = Image.open(img_path)
    # Read and prepare image
    img = image.load_img(img_path, target_size=(IMG_SIZE,IMG_SIZE,CHANNELS))
    img = image.img_to_array(img)
    img = img/255
    img = np.expand_dims(img, axis=0)
    
    fontsize = 1  # starting font size

    # portion of image width you want text width to be
    img_fraction = 0.85

    
    font = ImageFont.truetype("/usr/share/fonts/truetype/freefont/FreeMono.ttf", fontsize, encoding="unic")
    
    
    # Generate prediction
    data = model.predict(img)
    #print(data[0])
    conf = []
    for i in range(len(data[0])):
        if data[0][i]> 0.5:
            conf.append(data[0][i])
            #print(i, data[0][i])
    prediction = (model.predict(img) > 0.5).astype('int')
    #print('prediction_out :',prediction)
    prediction = pd.Series(prediction[0])
    
    mlb = MultiLabelBinarizer()
    mlb.fit(classes)

    # Loop over all labels and show them
    N_LABELS = len(mlb.classes_)
    prediction.index = mlb.classes_
    prediction = prediction[prediction==1].index.values
    label = str(prediction)
    conf = str(conf)
    txt = label + conf
    print('label : ', label)
    print('txt : '+txt )
    while font.getsize(txt)[0] < img_fraction*img1.size[0]:
    # iterate until the text size is just larger than the criteria
        fontsize += 1
        font = ImageFont.truetype("/usr/share/fonts/truetype/freefont/FreeMono.ttf", fontsize, encoding="unic")

    # optionally de-increment to be sure it is less than criteria
    fontsize -= 1
    font = ImageFont.truetype("/usr/share/fonts/truetype/freefont/FreeMono.ttf", fontsize, encoding="unic")
    
    
    draw = ImageDraw.Draw(img1)
    label_size = draw.textsize(txt)
    text_origin = tuple(np.array([15, 15 - label_size[1]]))
    print(text_origin)
    print(label_size[1])
    color = ImageColor.getrgb("green")
    # font = ImageFont.truetype(<font-file>, <font-size>)
    #font = ImageFont.truetype("/usr/share/fonts/truetype/freefont/FreeMono.ttf", 28, encoding="unic")
    # draw.text((x, y),"Sample Text",(r,g,b))
    #color = ImageColor.getrgb("green")
    draw.text(text_origin, txt, fill = color,font = font)
    filename = "output_multi/"+title
    img1.save(filename)

In [45]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from PIL import Image
from natsort import natsorted
from PIL import ImageFont
PATH_TO_TEST_IMAGES_DIR = 'multi_class_images'
image_name=[]

image_name=[f for f in listdir(PATH_TO_TEST_IMAGES_DIR) if isfile(os.path.join(PATH_TO_TEST_IMAGES_DIR, f))]
image_name = sorted(image_name)
image_name = natsorted(image_name)
#print(image_name)

titles = ["008e6476-ec4d-428a-ab68-69b022286d08_820291.jpg",
          "008e6476-ec4d-428a-ab68-69b022286d08_820299.jpg",
          "02c19447-c4c8-4945-9116-6d232d5308e7_819485.jpg",
          "03029226-766f-4abd-b5a2-28ede51bcced_819460.jpg",
          "92d0d2a9-6f02-43ac-a460-2aed85786a1a_821446.jpg",
          "69ff563b-ccc3-4c66-aaf3-64f4656e7a40_817463.jpg",
         "7ecf4700-492a-44fb-8335-e9b1f1e69104_820997.jpg",
         "a5081444-da42-472c-b1de-409d19322c3e_820142.jpg",
         "557c27d9-06cb-47ac-8067-b3a3c964cff5_819755.jpg",
         "365faade-07eb-4385-82bb-7e904c544006_819479.jpg",
         "d9d995b5-f134-427d-9ae1-b655397f00cf_819039.jpg"]

classes = [['Card', 'DigiLocker', 'Front','Page','back']]
for t in titles:
    show_prediction(t, model, classes)

label :  ['Card' 'Front']
txt : ['Card' 'Front'][0.9756422, 0.99900407]
(15, 4)
11
label :  ['Card' 'Front']
txt : ['Card' 'Front'][0.7648502, 0.9750734]
(15, 4)
11
label :  ['Card' 'Front']
txt : ['Card' 'Front'][0.967351, 0.99974483]
(15, 4)
11
label :  ['Card' 'Front']
txt : ['Card' 'Front'][0.9548307, 0.9995235]
(15, 4)
11
label :  ['Front' 'Page']
txt : ['Front' 'Page'][0.9962106, 0.9949662]
(15, 4)
11
label :  ['Card' 'Front']
txt : ['Card' 'Front'][0.96176857, 0.99972624]
(15, 4)
11
label :  ['Front' 'Page']
txt : ['Front' 'Page'][0.9976634, 0.6693771]
(15, 4)
11
label :  ['Card' 'back']
txt : ['Card' 'back'][0.9957489, 0.6382489]
(15, 4)
11
label :  ['Card' 'Front']
txt : ['Card' 'Front'][0.906886, 0.6366544]
(15, 4)
11
label :  ['Card' 'Front']
txt : ['Card' 'Front'][0.96703726, 0.9984744]
(15, 4)
11
label :  ['Card' 'back']
txt : ['Card' 'back'][0.7954242, 0.99978346]
(15, 4)
11


In [35]:
from sklearn.preprocessing import MultiLabelBinarizer
classes = [['Card', 'DigiLocker', 'Front','Page','back']]

print("Labels:")
mlb = MultiLabelBinarizer()
mlb.fit(classes)

# Loop over all labels and show them
N_LABELS = len(mlb.classes_)
print(mlb.classes_)
for (i, label) in enumerate(mlb.classes_):
    print("{}. {}".format(i, label))

Labels:
['Card' 'DigiLocker' 'Front' 'Page' 'back']
0. Card
1. DigiLocker
2. Front
3. Page
4. back


In [28]:
Card
1. DigiLocker
2. Front
3. Page
4. back

Geeks for Geeks
